In [16]:
import pandas as pd
import pymssql
import numpy as np
import re
import time
import heapq

### 主函数测试

#### 2021.9.18 优化了词表

In [53]:
def gettags_2(text, variety):
    word_lists = pd.read_csv('D:\Projects\新闻推荐\新闻推荐接口\get_tags\word_lists_72_20210918.csv')
    # word_lists = pd.read_csv('E:\\project\\djangoroot\\academicinfoapi\\get_tags\\word_lists_72_20210404.csv')
    db = pymssql.connect('47.104.142.34', 'xueshu', '@WSX3edc', 'AcademicExchangePlatform', charset='cp936')
    cursor = db.cursor()
    # LabelSort用不到
    sql = "select ObjectId,LabelSort,TagName,weight,relevancy_threshold from TB_Base_Tag where IsLastStage=1 order by LabelNum desc"
    cursor.execute(sql)
    wr = cursor.fetchall()  # 包含所有字段
    objectid = np.array(wr).T[0]
    labels = np.array(wr).T[2]
    weights = np.array(wr).T[3].astype(np.float)  # 矩阵转置并转换为float
    relevancy_threshold = np.array(wr).T[4].astype(np.float)
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = text.replace("\u3000", "")
    n = []
    len_text = len(text)
    unadjusted_len_text = len(text)
    if len_text < 800:
        pass
    elif len_text < 1000:
        len_text = len_text ** (39 / 40)
    else:
        len_text = len_text ** (14 / 15) + 211  # 211约等于1000^(39/40)-1000^(14/15)
    words_freq = []
    for i in range(0, word_lists.shape[1]):
        freq = 0
        matched_words=[]
        for word in word_lists.iloc[:, i]:
            if type(word) != str:
                continue
            if word.encode('utf-8').isalpha():  # 判断是否为纯英文，防止出现apple匹配pl的情况
                if text.find(word) == -1:
                    # if word in text: 慢的很
                    words_freq.append((word, 0))  # 改进算法速度，因为re正则匹配速度比较慢,太妙了
                else:
                    regex = re.compile(r'[^A-Za-z](' + word + ')[^A-Za-z]')
                    word_freq = len(regex.findall(text))
                    if word_freq:
                        freq += 1
                    words_freq.append((word, word_freq))
            else:
                word_freq = text.count(word)
                if word_freq != 0:
                    freq += 1
                    matched_words.append(word) #记录每个学科匹配的词，来优化词表
                words_freq.append((word, word_freq)) #放在上个if里面的话，如果只有不到5个词得到匹配就会报错
        n.append((i, freq, freq / len_text ,matched_words))  # freq/len_text
    freq_sum = np.dot(np.array(n).T[1], weights)  # 不同学科权重不同，感兴趣的学科freq_sum会有更大的权重，更容易相关
    k = freq_sum / unadjusted_len_text
    r = "相关" if k >= 0.33 else "不相关"
    # n.sort(key=lambda x: x[1], reverse=True) #各学科词表按匹配词数排序,应该按照相关性排！！！！！
    tags = []
    for t in n:
        correlated_rate = t[2] / relevancy_threshold[t[0]]  # 文章标签关联度
        if correlated_rate >= 1:  # 若比值大于阈值
            tags.append([labels[t[0]], 100 ,t[3],len(t[3])])
        elif correlated_rate >= 0.5:
            tags.append([labels[t[0]], correlated_rate * 100,t[3],len(t[3])])
        else:
            continue
    words_freq = list(set(words_freq))  # 去除重复值
    words_freq.sort(key=lambda x: x[1], reverse=True)
    keywords = words_freq[0:5]  # 此表中匹配次数前五的词
    db.close()
    cursor.close()
    if variety:  # 1为普通新闻 0为会议通知
        tags=pd.DataFrame(tags).nlargest(3,[1],keep='all').values.tolist() #返回前三高，并保留并列值
        return r, k, tags, keywords
    else:
        for label_name in labels:
            if text.find(label_name) != -1:
                for j in range(0, len(tags)):
                    if tags[j][0] == label_name:  # 布尔型索引,相同位置
                        tags[j][1] = 100  # 这里如果用for tag in tags，修改tag的值不会改变tags
                        break  # 为了提高效率break
                else:
                    tags.append([label_name, 100])  # for else语句 else会被break掉
        tags=pd.DataFrame(tags).nlargest(3,[1],keep='all').values.tolist()
        r = "相关"
        return r, k, tags, keywords

In [54]:
text='''
新凝胶可减少土地用水并避免污染

　俄罗斯托木斯克理工大学科研人员开发出一种由食物废料制成的凝胶。新材料可使农业生产中定点使用化肥和水，促进农业产业更加经济和环保。相关研究成果近日发表在《清洁生产杂志》上。

　　目前，水和肥料都没有在农业中得到最佳使用，传统的植物栽培方法，使大部分水和化学物质，包括对动物有毒的化学物质，穿过土壤“溜过”根部，与地下水和地表水混合。为此，全球农业科技工作者都在寻找可以解决上述问题的新材料。俄托木斯克理工大学科研人员开发出一种聚合物水凝胶。该凝胶可作为土壤的“智能”添加剂，有助于避免土壤污染，显著减少用水量并改善植物对肥料的吸收。

　　该大学化学与生物医学技术研究学院研究员安东尼奥·迪·马蒂诺说，新研发的材料能够在下雨或灌溉时储存大量水分，然后随着土壤变干缓慢释放。例如，在一些地区使用这种凝胶，可将灌溉次数减少到每周一次。他还称，新研发的水凝胶完全可以生物降解，也就是说，经过一段时间后，它们会“溶解”在土壤中，毫无踪迹。

　　安东尼奥·迪·马蒂诺表示，新技术的一个关键特征是能够用农业和食品工业的有机废料生产凝胶，其主要材料是多糖和蛋白质。但科研人员也在研究纤维素、海藻酸和其他一些生物聚合物的可能性，尽量减少或完全避免使用化学品，并避免采纳需要大量能源的流程。

　　据悉，新的水凝胶可用于更高效、更温和地施用肥料或其他化学品，用添加剂浸透后的成品凝胶只需简单地分置在植物根部周围就可以，其使用寿命取决于凝胶的具体成分，平均为几个星期，这种凝胶适用于大型农场和园艺业。
'''
print(gettags_2(text,1))

d:\projects\新闻推荐\新闻推荐接口\venv\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (0,7,8,9,11,12,17,18,19,21,26,27,31,33,38,44,49,50,52,55,57,58,59,62) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
d:\projects\新闻推荐\新闻推荐接口\venv\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
d:\projects\新闻推荐\新闻推荐接口\venv\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modi

('相关', 0.7984375, [['精细化工', 71.42857142857143, ['生物降解', '添加剂', '水凝胶', '清洁生产', '纤维素', '蛋白质', '农业生产', '聚合物', '化学物质', '土壤污染', '化学品', '糖', '新材料', '水分', '地下水', '生物医学'], 16], ['环境化学', 62.499999999999986, ['地下水', '生物降解', '添加剂', '土壤污染', '化学物质', '纤维素', '土壤', '蛋白质', '地表水', '聚合物', '化学品', '生物聚合', '食品工业', '污染'], 14], ['高分子化学', 58.03571428571428, ['水凝胶', '纤维素', '聚合物', '生物降解', '清洁生产', '添加剂', '蛋白质', '食品工业', '糖', '土壤污染', '海藻酸', '地下水', '化学品'], 13], ['环境生态与毒理学', 58.03571428571428, ['生物降解', '地下水', '土壤污染', '化学品', '土壤', '化学物质', '蛋白质', '纤维素', '水凝胶', '添加剂', '食品工业', '藻', '污染'], 13], ['三废', 58.03571428571428, ['清洁生产', '地下水', '土壤污染', '生物降解', '化学物质', '添加剂', '蛋白质', '纤维素', '聚合物', '化学品', '水凝胶', '新材料', '污染'], 13]], [('土壤', 5), ('农业', 5), ('肥料', 3), ('水凝胶', 3), ('添加剂', 2)])


#### 2021.9.19 算法改进
当多个标签得分相同时，返回得分前三名的所有标签

In [45]:
[tttt[i] for i in pd.DataFrame(tttt).nlargest(3,[1],keep='all').index]

[['精细化工',
  71.42857142857143,
  ['生物降解',
   '添加剂',
   '水凝胶',
   '清洁生产',
   '纤维素',
   '蛋白质',
   '农业生产',
   '聚合物',
   '化学物质',
   '土壤污染',
   '化学品',
   '糖',
   '新材料',
   '水分',
   '地下水',
   '生物医学'],
  16],
 ['环境化学',
  62.499999999999986,
  ['地下水',
   '生物降解',
   '添加剂',
   '土壤污染',
   '化学物质',
   '纤维素',
   '土壤',
   '蛋白质',
   '地表水',
   '聚合物',
   '化学品',
   '生物聚合',
   '食品工业',
   '污染'],
  14],
 ['高分子化学',
  58.03571428571428,
  ['水凝胶',
   '纤维素',
   '聚合物',
   '生物降解',
   '清洁生产',
   '添加剂',
   '蛋白质',
   '食品工业',
   '糖',
   '土壤污染',
   '海藻酸',
   '地下水',
   '化学品'],
  13],
 ['环境生态与毒理学',
  58.03571428571428,
  ['生物降解',
   '地下水',
   '土壤污染',
   '化学品',
   '土壤',
   '化学物质',
   '蛋白质',
   '纤维素',
   '水凝胶',
   '添加剂',
   '食品工业',
   '藻',
   '污染'],
  13],
 ['三废',
  58.03571428571428,
  ['清洁生产',
   '地下水',
   '土壤污染',
   '生物降解',
   '化学物质',
   '添加剂',
   '蛋白质',
   '纤维素',
   '聚合物',
   '化学品',
   '水凝胶',
   '新材料',
   '污染'],
  13]]

In [50]:
pd.DataFrame(tttt).nlargest(3,[1],keep='all').values.tolist()

[['精细化工',
  71.42857142857143,
  ['生物降解',
   '添加剂',
   '水凝胶',
   '清洁生产',
   '纤维素',
   '蛋白质',
   '农业生产',
   '聚合物',
   '化学物质',
   '土壤污染',
   '化学品',
   '糖',
   '新材料',
   '水分',
   '地下水',
   '生物医学'],
  16],
 ['环境化学',
  62.499999999999986,
  ['地下水',
   '生物降解',
   '添加剂',
   '土壤污染',
   '化学物质',
   '纤维素',
   '土壤',
   '蛋白质',
   '地表水',
   '聚合物',
   '化学品',
   '生物聚合',
   '食品工业',
   '污染'],
  14],
 ['高分子化学',
  58.03571428571428,
  ['水凝胶',
   '纤维素',
   '聚合物',
   '生物降解',
   '清洁生产',
   '添加剂',
   '蛋白质',
   '食品工业',
   '糖',
   '土壤污染',
   '海藻酸',
   '地下水',
   '化学品'],
  13],
 ['环境生态与毒理学',
  58.03571428571428,
  ['生物降解',
   '地下水',
   '土壤污染',
   '化学品',
   '土壤',
   '化学物质',
   '蛋白质',
   '纤维素',
   '水凝胶',
   '添加剂',
   '食品工业',
   '藻',
   '污染'],
  13],
 ['三废',
  58.03571428571428,
  ['清洁生产',
   '地下水',
   '土壤污染',
   '生物降解',
   '化学物质',
   '添加剂',
   '蛋白质',
   '纤维素',
   '聚合物',
   '化学品',
   '水凝胶',
   '新材料',
   '污染'],
  13]]

### isactivity测试
### 正则表达式测试

In [ ]:
def isActivity(title):
    f=open('无关通讯过滤.txt','r',encoding='utf-8')
    lines=f.readlines()
    for word in [i.strip('\n') for i in lines]:
        regex = re.compile(word)
        if regex.search(title):
            f.close()
            return 1 #1为活动，国家级以下获奖类新闻
    f.close()
    return 0

In [92]:
t1 = "四川省能源中心获四川省牛人奖"
p1 = "\S\S{2,3}[省市州]\S+[奖]" 
t2="天大深圳学院绿色建筑与海绵城市科研中心部署今年工作"
p2="部署\S+工作"
t3="天津大学助力河南创新驱动发展 建设中原先进技术研究院"
p3="(组建|建设)\S+(实验室|研究院)"
regex = re.compile(p3)
result=regex.search(t3)
print(result.group(1))

建设


In [104]:
testdata=pd.read_excel('文章标题总结.xlsx',header=None)
for i in testdata.iterrows():
    print(i[0],isActivity(i[1][0]))

0 1
1 1
2 0
3 1
4 0
5 1
6 1
7 1
8 1
9 0
10 1
11 0
12 1
13 1
14 0
15 1
16 1
17 1
18 1
19 1
20 0
21 1
22 1
23 0
24 1
25 1
26 0
27 0
28 1
29 1
30 0
31 1
32 1
33 1


### 算法改进
对于英文单词用正则表达式匹配 因为有word边界的问题

In [ ]:
zz='LED'
ttt='阿克苏,LED凯撒'
regex=re.compile(r'[^A-Za-z]('+zz+')[^A-Za-z]')
fffffffff=regex.findall(ttt)

### 其他

In [ ]:
import math
value=1000
value ** (1 / 1000)
math.log(100,1.0046157902783952) #log函数不行，因为一开始上涨很快，斜率
1000 ** (39/40)-(1000 ** (14/15))

In [12]:
#
#tag_weight和TB_base_tag整合
for i in range(0,72):
    weight=float(wr[i][3])
    relevancy_threshold=float(wr[i][4])
    objectid=wr[i][0]
    sql="update TB_Base_Tag set weight= %s,relevancy_threshold=%s where ObjectId=%s"
    cursor.execute(sql,(weight,relevancy_threshold,objectid))
    db.commit()

In [13]:
#老版本
def gettags(text):
    word_lists = pd.read_csv('D:\Projects\新闻推荐\新闻推荐接口\get_tags\word_lists_72.csv')
    db = pymssql.connect('47.104.142.34', 'xueshu',   '@WSX3edc', 'AcademicExchangePlatform')
    cursor = db.cursor()
    sql = "select * from Tag_weight"
    cursor.execute(sql)
    wr = cursor.fetchall()  # 包含所有字段
    weights = np.array(wr).T[3].astype(np.float)  # 矩阵转置并转换为float
    relevancy_threshold = np.array(wr).T[4].astype(np.float)
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = text.replace("\u3000", "")
    n = []
    len_text = len(text)
    words_freq = []
    for i in range(0, word_lists.shape[1]):
        freq = 0
        for word in word_lists.iloc[:, i]:
            if type(word) != str:
                continue
            if word.encode('utf-8').isalpha():  # 判断是否为纯英文，防止出现apple匹配pl的情况
                regex = re.compile(r'[^A-Za-z](' + word + ')[^A-Za-z]')
                find = regex.findall(text)
                word_freq = len(find)
                if find:
                    freq += 1
                words_freq.append((word, word_freq))
            else:
                word_freq = text.count(word)
                if word_freq != 0:
                    freq += 1
                words_freq.append((word, word_freq))
        n.append((i, freq, freq / len_text))  # freq/len_text
    freq_sum = np.dot(np.array(n).T[1], weights)
    k = freq_sum / len_text
    r = "相关" if k >= 0.33 else "不相关"
    n.sort(key=lambda x: x[1], reverse=True)
    tags = []
    for i in n[0:3]:
        correlated_rate = i[2] / relevancy_threshold[i[0]]  # 文章标签关联度
        if correlated_rate >= 1:  # 若比值大于阈值
            tags.append((wr[i[0]][0], 100))
        elif correlated_rate >= 0.5:
            tags.append((wr[i[0]][0], correlated_rate * 100))
        else:
            continue
    words_freq = list(set(words_freq))  # 去除重复值
    words_freq.sort(key=lambda x: x[1], reverse=True)
    keywords = words_freq[0:5]  # 此表中匹配次数前五的词
    db.close()
    cursor.close()
    return r, tags, keywords


